In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, make_scorer
from cost_function import cost_function, cost_function_cutoff

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key_train = 'In_Class_Assignments/turnover_train.csv'
file_key_val = 'In_Class_Assignments/turnover_val.csv'
file_key_test = 'In_Class_Assignments/turnover_test.csv'

bucket_object_train = bucket.Object(file_key_train)
file_object_train = bucket_object_train.get()
file_content_stream_train = file_object_train.get('Body')

bucket_object_val = bucket.Object(file_key_val)
file_object_val = bucket_object_val.get()
file_content_stream_val = file_object_val.get('Body')

bucket_object_test = bucket.Object(file_key_test)
file_object_test = bucket_object_test.get()
file_content_stream_test = file_object_test.get('Body')

## Reading the csv file
train = pd.read_csv(file_content_stream_train)
validation = pd.read_csv(file_content_stream_val)
test = pd.read_csv(file_content_stream_test)

In [2]:
## Changing sales to dummy variables
train = pd.concat([train.drop(columns = ['sales'], axis = 1), pd.get_dummies(train['sales'])], axis = 1)
validation = pd.concat([validation.drop(columns = ['sales'], axis = 1), pd.get_dummies(validation['sales'])], axis = 1)
test = pd.concat([test.drop(columns = ['sales'], axis = 1), pd.get_dummies(test['sales'])], axis = 1)

## Changing salary to dummy variables
train = pd.concat([train, pd.get_dummies(train['salary'])], axis = 1)
validation = pd.concat([validation, pd.get_dummies(validation['salary'])], axis = 1)
test = pd.concat([test, pd.get_dummies(test['salary'])], axis = 1)

In [3]:
## Creating interactions/features from the decision tree
train['interaction_1'] = np.where((train['satisfaction_level'] <= 0.465) & (train['number_project'] <= 2.5) & (train['last_evaluation'] <= 0.575), 1, 0)
train['interaction_2'] = np.where((train['satisfaction_level'] <= 0.465) & (train['number_project'] >= 2.5) & (train['satisfaction_level'] >= 0.115), 1, 0)
train['interaction_3'] = np.where((train['satisfaction_level'] >= 0.465) & (train['time_spend_company'] <= 4.5) & (train['average_montly_hours'] <= 290.5), 1, 0)

validation['interaction_1'] = np.where((validation['satisfaction_level'] <= 0.465) & (validation['number_project'] <= 2.5) & (validation['last_evaluation'] <= 0.575), 1, 0)
validation['interaction_2'] = np.where((validation['satisfaction_level'] <= 0.465) & (validation['number_project'] >= 2.5) & (validation['satisfaction_level'] >= 0.115), 1, 0)
validation['interaction_3'] = np.where((validation['satisfaction_level'] >= 0.465) & (validation['time_spend_company'] <= 4.5) & (validation['average_montly_hours'] <= 290.5), 1, 0)

test['interaction_1'] = np.where((test['satisfaction_level'] <= 0.465) & (test['number_project'] <= 2.5) & (test['last_evaluation'] <= 0.575), 1, 0)
test['interaction_2'] = np.where((test['satisfaction_level'] <= 0.465) & (test['number_project'] >= 2.5) & (test['satisfaction_level'] >= 0.115), 1, 0)
test['interaction_3'] = np.where((test['satisfaction_level'] >= 0.465) & (test['time_spend_company'] <= 4.5) & (test['average_montly_hours'] <= 290.5), 1, 0)

In [4]:
## Defining the input and target variables
X_train = train[['interaction_3', 'interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_train = train['left']

X_val = validation[['interaction_3', 'interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_val = validation['left']

X_test = test[['interaction_3', 'interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
Y_test = test['left']

## Changing the scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

## Random Forest

In [5]:
## Defining the hyper-parameters
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split' : [10, 15],
                 'min_samples_leaf' : [5, 7],
                 'max_depth' : [3, 5, 7]}

## Defining the customized scoring function 
my_score_function = make_scorer(cost_function, greater_is_better = True, needs_proba = True)

## Performing grid search
RF_grid_search = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

## Extraciting the best model 
RF_md = RF_grid_search.best_estimator_

In [7]:
## Predicting on validation and test 
RF_val_pred = RF_md.predict_proba(X_val)[:, 1]
RF_test_pred = RF_md.predict_proba(X_test)[:, 1]

## Identifying the optimal cut-off value
opt_cutoff = cost_function_cutoff(Y_val, RF_val_pred)

## Changing the likleihoods to labels
RF_label = np.where(RF_test_pred < opt_cutoff, 0, 1)

## Computing the confusion matrix
X = confusion_matrix(Y_test, RF_label)
print(X)
print('The cost of the RF model is:', -1500*X[1,0] - 1000*X[0, 1] + 500*X[1, 1])

[[1130   13]
 [  28  329]]
The cost of the RF model is: 109500


## Support Vector Machine 

In [8]:
## Defining the hyper-parameters
SVM_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'],
                  'C': [0.01, 0.1, 1, 10],
                  'gamma': [0.01, 0.1, 1]}

## Performing grid search
svm_grid_search = GridSearchCV(SVC(probability = True), SVM_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_train, Y_train)

## Extraciting the best model 
svm_md = svm_grid_search.best_estimator_

In [9]:
## Predicting on validation and test 
svm_val_pred = svm_md.predict_proba(X_val)[:, 1]
svm_test_pred = svm_md.predict_proba(X_test)[:, 1]

## Identifying the optimal cut-off value
opt_cutoff = cost_function_cutoff(Y_val, svm_val_pred)

## Changing the likleihoods to labels
svm_label = np.where(svm_test_pred < opt_cutoff, 0, 1)

## Computing the confusion matrix
X = confusion_matrix(Y_test, svm_label)
print(X)
print('The cost of the SVM model is:', -1500*X[1,0] - 1000*X[0, 1] + 500*X[1, 1])

[[1121   22]
 [  33  324]]
The cost of the SVM model is: 90500


## Ensemble

In [10]:
## Building the ensemble
X_ensemble = pd.concat([pd.DataFrame(RF_val_pred), pd.DataFrame(svm_val_pred)], axis = 1)

## Performing grid search with 3 folds 
RF_grid_search_ensemble = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv = 3, scoring = my_score_function, n_jobs = -1).fit(X_ensemble, Y_val)

## Extracting the best model 
RF_md_ensemble = RF_grid_search_ensemble.best_estimator_

## Identifying the optimal cutoff
opt_cutoff = cost_function_cutoff(Y_val, RF_md_ensemble.predict_proba(X_ensemble)[:, 1])

## Consolidating prediction for the ensemble
X = pd.concat([pd.DataFrame(RF_test_pred), pd.DataFrame(svm_test_pred)], axis = 1)

## Predicting 
ensemble_pred = RF_md_ensemble.predict_proba(X)[:, 1]
ensemble_label = np.where(ensemble_pred < opt_cutoff, 0, 1)

## Creating the confusion matrix 
X = confusion_matrix(Y_test, ensemble_label)
print(X)
print('The cost of the ensemble model is:', -1500*X[1,0] - 1000*X[0, 1] + 500*X[1, 1])

[[1132   11]
 [  33  324]]
The cost of the ensemble model is: 101500


In [ ]:
## Based on my result, I would use the random forest model to predict left.